# Getting distance following the reads

There are several libraries for different programming languages, they are listed [here on the OpenStreetMap wiki](https://wiki.openstreetmap.org/wiki/Routing#Developers).

Because we are using mainly the Python language, I will first try Python libraries. However, if they prove to be too slow, I will check other languages.

For Python, two libraries are given :
- [PyrouteLib](https://wiki.openstreetmap.org/wiki/PyrouteLib)
- [SimpleOsmRouter](https://github.com/F6F/SimpleOsmRouter)

> SimpleOsmRouter being far too simple, I will not cover it here

## PyrouteLib

### ~~Getting the *OSM* file~~

~~The library needs to access an OSM file containing a map of the area to be able to create a route.
When the Nomitatim server will be ready, we will be able to use the data there directly, but for now, I will use an extract containing only the city of London. A list of several extracts OSM files can be found [here](https://download.geofabrik.de/).~~

### Getting the library

First, the python package `osmapi` is needed, you can install it with `conda` or `pip`.
Then, the files present in the repository didn't work directly, I had to modify bit (mostly the `import` paths). I uploaded the fixed version of the library along with this notebook.

### Computing the route and distance from A to B


First, we get the coordinates of *A* and *B* :

In [1]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Test")

location = geolocator.geocode("Prittwitzstrasse, Ulm")
#location = geolocator.geocode("221B Baker street, London")
a = (location.latitude, location.longitude)

#location = geolocator.geocode("Trafalgar Square, London")
location = geolocator.geocode("Albert-Einstein-allee, Ulm")
b = (location.latitude, location.longitude)

Then, we use the library to get the different points that make the route and convert them to GPS coordinates :

In [2]:
from pyroutelib2.loadOsm import LoadOsm
from pyroutelib2.route import Router

# By default, it uses the open API, this can be changed directly in the file `loadOsm.py`.
# Here, we use bicycle to calculate routes.
data = LoadOsm("cycle")
router = Router(data)

node_a = data.findNode(a[0], a[1])
node_b = data.findNode(b[0], b[1])

result, route = router.doRoute(node_a, node_b)
lats = []
lons = []
if result == 'success':
    for i in route:
        node = data.rnodes[i]
        lats.append(node[0])
        lons.append(node[1])
        # print("%d: %f,%f" % (i, node[0], node[1]))
else:
    print("Error calculating the route.")

ParseError: unclosed token: line 78725, column 2 (<string>)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.plot(lons, lats)

Finally we can compute the distance using the distance *"as the crow flies"* between each two points :

In [ ]:
from geopy import distance

distance_route = 0

for i in range(len(lons)-1):
    p1=(lats[i],lons[i])
    p2=(lats[i+1],lons[i+1])
    distance_route += distance.geodesic(p1,p2, ellipsoid='GRS-80').km

print("Total distance on the route : %.2fkm" % distance_route)

### Comparison with Google Maps

#### Trafalgar Square to 221B Baker Street, London

By using *Pyroutelib*, we get a distance by route of **3.82km**. Google Maps gives a similar result of **3.6km** :

![Google Map itinary from Trafalgar Square to 221B Baker street in London](./img/Trafalgar_Square_to_221B_Baker_St.png)


#### Prittwitzstraße to Albert-Einstein-Allee, Ulm

By using *Pyroutelib*, we get a distance by route of **5.87km**. Google Maps gives a similar result of **5.2km** :

![Google Map itinary from Prittwitzstraße to Albert-Einstein-Allee in Ulm](./img/Prittwitzstrasse_AlbertEinsteinAlle.png)

### Note on performance

I tried to calculate the route between **Rennes, France** and **Brest, France** and it took nearly an hour, but is is quite fast for shorter distance (like here inside London or Ulm). We will maybe have to look at **compiled** languages libraries.